<a href="https://colab.research.google.com/github/tera90223/mit-map/blob/main/MIT_Problem_Set_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access problem set
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Problem Set 2: Finding shortest paths through MIT buildings

## Graph Data Structures

In [4]:
# 6.0002 Problem Set 5
# Graph optimization
# Name: Chantera Lazard

import unittest

#
# A set of data structures to represent graphs
#

class Node(object):
    """Represents a node in the graph"""
    def __init__(self, name):
        self.name = str(name)

    def get_name(self):
        return self.name

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return self.name == other.name

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        # This function is necessary so that Nodes can be used as
        # keys in a dictionary, even though Nodes are mutable
        return self.name.__hash__()


class Edge(object):
    """Represents an edge in the dictionary. Includes a source and
    a destination."""
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest

    def get_source(self):
        return self.src

    def get_destination(self):
        return self.dest

    def __str__(self):
        return '{}->{}'.format(self.src, self.dest)


class WeightedEdge(Edge):
    def __init__(self, src, dest, total_distance, outdoor_distance):
        # DRY Principle
        super().__init__(src, dest)
        self.total_distance = total_distance
        self.outdoor_distance = outdoor_distance

    def get_total_distance(self):
        return self.total_distance

    def get_outdoor_distance(self):
        return self.outdoor_distance

    def __str__(self):
        return '{}->{} ({}, {})'.format(self.src, self.dest, \
                                        self.total_distance, \
                                        self.outdoor_distance)


class Digraph(object):
    """Represents a directed graph of Node and Edge objects"""
    def __init__(self):
        self.nodes = set([])
        self.edges = {}  # must be a dict of Node -> list of edges

    def __str__(self):
        edge_strs = []
        for edges in self.edges.values():
            for edge in edges:
                edge_strs.append(str(edge))
        edge_strs = sorted(edge_strs)  # sort alphabetically
        return '\n'.join(edge_strs)  # concat edge_strs with "\n"s between them

    def get_edges_for_node(self, node):
        return self.edges[node]

    def has_node(self, node):
        return node in self.nodes

    def add_node(self, node):
        """Adds a Node object to the Digraph. Raises a ValueError if it is
        already in the graph."""
        if self.has_node(node):
            raise ValueError('Node already in graph')
        else:
            self.nodes.add(node)
            self.edges[node] = []

    def add_edge(self, edge):
        """Adds an Edge or WeightedEdge instance to the Digraph. Raises a
        ValueError if either of the nodes associated with the edge is not
        in the  graph."""
        src = edge.get_source()
        dest = edge.get_destination()
        if not(self.has_node(src) and self.has_node(dest)):
            raise ValueError('Node not in graph')
        self.edges[src].append(edge)


In [ ]:
# ================================================================
# Begin tests -- you do not need to modify anything below this line
# ================================================================

class TestGraph(unittest.TestCase):

    def setUp(self):
        self.g = Digraph()
        self.na = Node('a')
        self.nb = Node('b')
        self.nc = Node('c')
        self.g.add_node(self.na)
        self.g.add_node(self.nb)
        self.g.add_node(self.nc)
        self.e1 = WeightedEdge(self.na, self.nb, 15, 10)
        self.e2 = WeightedEdge(self.na, self.nc, 14, 6)
        self.e3 = WeightedEdge(self.nb, self.nc, 3, 1)
        self.g.add_edge(self.e1)
        self.g.add_edge(self.e2)
        self.g.add_edge(self.e3)

    def test_weighted_edge_str(self):
        self.assertEqual(str(self.e1), "a->b (15, 10)")
        self.assertEqual(str(self.e2), "a->c (14, 6)")
        self.assertEqual(str(self.e3), "b->c (3, 1)")

    def test_weighted_edge_total_distance(self):
        self.assertEqual(self.e1.get_total_distance(), 15)
        self.assertEqual(self.e2.get_total_distance(), 14)
        self.assertEqual(self.e3.get_total_distance(), 3)

    def test_weighted_edge_outdoor_distance(self):
        self.assertEqual(self.e1.get_outdoor_distance(), 10)
        self.assertEqual(self.e2.get_outdoor_distance(), 6)
        self.assertEqual(self.e3.get_outdoor_distance(), 1)

    def test_add_edge_to_nonexistent_node_raises(self):
        node_not_in_graph = Node('q')
        no_src = WeightedEdge(self.nb, node_not_in_graph, 5, 5)
        no_dest = WeightedEdge(node_not_in_graph, self.na, 5, 5)

        with self.assertRaises(ValueError):
            self.g.add_edge(no_src)
        with self.assertRaises(ValueError):
            self.g.add_edge(no_dest)

    def test_add_existing_node_raises(self):
        with self.assertRaises(ValueError):
            self.g.add_node(self.na)

    def test_graph_str(self):
        expected = "a->b (15, 10)\na->c (14, 6)\nb->c (3, 1)"
        self.assertEqual(str(self.g), expected)


if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.009s

OK


## Building the Campus Map




## Designing Your Graph



1.   What do the graph's nodes represent in this problem?
  *   Nodes represent MIT buildings identified by a unique number.

2.    What do the graph's edges represent?
  *   Graph edge's represent a directed path between two buildings
3.    Where are the distances represented?
  *   Each edge stores two types of distance values:
        1. Total Distance - the full length of the path between two buildings.
        2. Outdoor Distance - the portion of that path which is outdoors.
      
      The outdoor distance is always less than or equal to the total distance.






### Implementing load_map

In [5]:
# 6.0002 Problem Set 5
# Graph optimization
# Name: Chantera Lazard

from google.colab import userdata
#
# Problem 2: Building up the Campus Map
#

# Problem 2b: Implementing load_map
def load_map(map_filename):
    """
    Parses the map file and constructs a directed graph

    Parameters:
        map_filename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four positive
        integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a Digraph representing the map
    """

    print("Loading map from file...")
    # Create the Digraph Object
    g = Digraph()

    # Open the map file
    with open(map_filename, 'r') as file:
        for line in file:
          #initialize the buildings and distances
          src, dest, total_distance, outdoor_distance = line.split(' ')
          total_distance = int(total_distance)
          outdoor_distance = int(outdoor_distance)

          # Create nodes if not already loaded in graph
          if not g.has_node(Node(src)):
            g.add_node(Node(src))
          if not g.has_node(Node(dest)):
            g.add_node(Node(dest))

          # Create weighted edge and add it in graph
          w_edge = WeightedEdge(Node(src), Node(dest), total_distance, outdoor_distance)
          g.add_edge(w_edge)
          print("adding edge....", str(w_edge))

    return g

### Testing load_map

In [6]:
# Problem 2c: Testing load_map
# Test load_map with a test file
g = load_map(userdata.get('test_load_map.txt'))


Loading map from file...
adding edge.... A->B (10, 9)
adding edge.... A->C (12, 2)
adding edge.... B->C (1, 1)


## Finding the Shortest Path Using Optimized Search Method

### Objective Function


*   What is the objective function for this problem? What are the constraints?
    *   **Minimize total distance traveled** between the `start` and `end` nodes.
    *    **Constraints:**
         * The **outdoor distance** of the path must not exceed `max_dist_outdoors`
         * Find the path with the **least total distance traveled**
         * The edges are **directed**
         * The path will **not** contain *cycles* - revisit nodes in the current path



### Implement get_best_path

In [8]:
# Problem 3b: Implement get_best_path
def get_best_path(digraph, start, end, path, max_dist_outdoors, best_dist,
                  best_path):
    """
    Finds the shortest path between buildings subject to constraints.

    Parameters:
        digraph: Digraph instance
            The graph on which to carry out the search
        start: string
            Building number at which to start
        end: string
            Building number at which to end
        path: list composed of [[list of strings], int, int]
            Represents the current path of nodes being traversed. Contains
            a list of node names, total distance traveled, and total
            distance outdoors.
        max_dist_outdoors: int
            Maximum distance spent outdoors on a path
        best_dist: int
            The smallest distance between the original start and end node
            for the initial problem that you are trying to solve
        best_path: list of strings
            The shortest path found so far between the original start
            and end node.

    Returns:
        A tuple with the shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k and the distance of that path.

        If there exists no path that satisfies max_total_dist and
        max_dist_outdoors constraints, then return None.
    """
    # TODO
    # Check if start and end nodes are in the graph, if not raise ValueError
    if not(Node(start) in digraph.nodes and Node(end) in digraph.nodes):
      raise ValueError('Start or end node not in graph')
    # If path is empty, initialize path
    if path == []:
      path = [[start], 0, 0]
    # Base Case: Check if the nodes are the same, return the current path
    # and its total_distance
    if start == end:
     return path[0], path[1]
    else:
      # Transverse all outgoing edges from the start node to its children
      for edge in digraph.get_edges_for_node(Node(start)):
        # Get a child node
        current_node = edge.get_destination().get_name()
        # Make sure node is not apart of path to prevent cycles
        if current_node not in path[0]:
          # Create new_path with updated node list and updated distances
          new_path = [path[0] + [current_node], \
                     path[1] + edge.get_total_distance(), \
                     path[2] + edge.get_outdoor_distance()]
          # If this is the first valid path (best_dist is None), OR
          # this new path is shorter than the current best path AND,
          # it satisfies the outdoor constraint,
          # recursively explore deeper using the current child as the new start
          if (best_dist is None or new_path[1] < best_dist) \
              and new_path[2] <= max_dist_outdoors:
            result = get_best_path(digraph, current_node, end, new_path, \
                                    max_dist_outdoors,\
                                    best_dist, best_path)
            # If a valid result is retuned, update the best path and best
            # distance found so far
            if result is not None:
              best_path = result[0]
              best_dist = result[1]
    return (best_path, best_dist)

### Implement directed_dfs

In [2]:
# Problem 3c: Implement directed_dfs
def directed_dfs(digraph, start, end, max_total_dist, max_dist_outdoors):
    """
    Finds the shortest path from start to end using a directed depth-first
    search. The total distance traveled on the path must not
    exceed max_total_dist, and the distance spent outdoors on this path must
    not exceed max_dist_outdoors.

    Parameters:
        digraph: Digraph instance
            The graph on which to carry out the search
        start: string
            Building number at which to start
        end: string
            Building number at which to end
        max_total_dist: int
            Maximum total distance on a path
        max_dist_outdoors: int
            Maximum distance spent outdoors on a path

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k

        If there exists no path that satisfies max_total_dist and
        max_dist_outdoors constraints, then raises a ValueError.
    """
    # TODO
    best_path, total_dist = get_best_path(digraph=digraph, start=start, end=end, \
                                          path=[], \
                                          max_dist_outdoors=max_dist_outdoors, \
                                          best_dist = None,
                                          best_path = None)
    if best_path == None or total_dist > max_total_dist:
      raise ValueError('No path found within distance provided')
    return best_path

### Test get_best_path and directed_dfs

## 💡 Reflections on Recursive Pathfinding

This function taught me some key truths about recursion — not just in theory, but in messy, code-level reality.

### 🔁 1. Recursive Calls Still Return

Recursion is a function calling itself, yes — but like any function, it must **return a result** to be useful. The base case is where recursion stops and begins bubbling values back up the call stack. That’s why you must:
- Handle the `start == end` case cleanly
- Return a path and distance from it and not just "hit it and quit it"

### 🧠 2. Don't Mutate Shared State

Each recursive branch represents a **new possibility**, not a continuation of the same state. So:
- Rebuild `new_path` instead of mutating `path`
- Track total and outdoor distance **per path**, not globally
- Treat recursion like building a timeline where every fork needs its own copy

### 💥 3. Return Values Are Not Side Effects

You can’t just hope a recursive call "does the work" — it returns a value, and you need to catch it. That’s why we assign the result like this:

```python result = get_best_path(...)```

### 🚨 4. Python Will Punish Unclear Scope

If a variable like `new_path` is conditionally created and you try to use it outside its block, python will break. This problem reminded me that:
- Order matters
- Identation is scope
- Initialization must happen before use


In [9]:
# ================================================================
# Begin tests -- you do not need to modify anything below this line
# ================================================================

class Ps2Test(unittest.TestCase):
    LARGE_DIST = 99999

    def setUp(self):
        self.graph = load_map(userdata.get("mit_map.txt"))

    def test_load_map_basic(self):
        self.assertTrue(isinstance(self.graph, Digraph))
        self.assertEqual(len(self.graph.nodes), 37)
        all_edges = []
        for _, edges in self.graph.edges.items():
            all_edges += edges  # edges must be dict of node -> list of edges
        all_edges = set(all_edges)
        self.assertEqual(len(all_edges), 129)

    def _print_path_description(self, start, end, total_dist, outdoor_dist):
        constraint = ""
        if outdoor_dist != Ps2Test.LARGE_DIST:
            constraint = "without walking more than {}m outdoors".format(
                outdoor_dist)
        if total_dist != Ps2Test.LARGE_DIST:
            if constraint:
                constraint += ' or {}m total'.format(total_dist)
            else:
                constraint = "without walking more than {}m total".format(
                    total_dist)

        print("------------------------")
        print("Shortest path from Building {} to {} {}".format(
            start, end, constraint))

    def _test_path(self,
                   expectedPath,
                   total_dist=LARGE_DIST,
                   outdoor_dist=LARGE_DIST):
        start, end = expectedPath[0], expectedPath[-1]
        self._print_path_description(start, end, total_dist, outdoor_dist)
        dfsPath = directed_dfs(self.graph, start, end, total_dist, outdoor_dist)
        print("Expected: ", expectedPath)
        print("DFS: ", dfsPath)
        self.assertEqual(expectedPath, dfsPath)

    def _test_impossible_path(self,
                              start,
                              end,
                              total_dist=LARGE_DIST,
                              outdoor_dist=LARGE_DIST):
        self._print_path_description(start, end, total_dist, outdoor_dist)
        with self.assertRaises(ValueError):
            directed_dfs(self.graph, start, end, total_dist, outdoor_dist)

    def test_path_one_step(self):
        self._test_path(expectedPath=['32', '56'])

    def test_path_no_outdoors(self):
        self._test_path(
            expectedPath=['32', '36', '26', '16', '56'], outdoor_dist=0)

    def test_path_multi_step(self):
        self._test_path(expectedPath=['2', '3', '7', '9'])

    def test_path_multi_step_no_outdoors(self):
        self._test_path(
            expectedPath=['2', '4', '10', '13', '9'], outdoor_dist=0)

    def test_path_multi_step2(self):
        self._test_path(expectedPath=['1', '4', '12', '32'])

    def test_path_multi_step_no_outdoors2(self):
        self._test_path(
            expectedPath=['1', '3', '10', '4', '12', '24', '34', '36', '32'],
            outdoor_dist=0)

    def test_impossible_path1(self):
        self._test_impossible_path('8', '50', outdoor_dist=0)

    def test_impossible_path2(self):
        self._test_impossible_path('10', '32', total_dist=100)


if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

.

Loading map from file...
adding edge.... 32->36 (70, 0)
adding edge.... 36->32 (70, 0)
adding edge.... 32->57 (30, 0)
adding edge.... 57->32 (30, 0)
adding edge.... 32->76 (80, 50)
adding edge.... 76->32 (80, 50)
adding edge.... 32->68 (110, 80)
adding edge.... 68->32 (110, 80)
adding edge.... 76->68 (72, 30)
adding edge.... 68->76 (72, 30)
adding edge.... 68->66 (51, 0)
adding edge.... 66->68 (51, 0)
adding edge.... 68->56 (80, 70)
adding edge.... 56->68 (80, 70)
adding edge.... 66->56 (40, 0)
adding edge.... 56->66 (40, 0)
adding edge.... 76->66 (130, 100)
adding edge.... 66->76 (130, 100)
adding edge.... 56->18 (35, 0)
adding edge.... 18->56 (35, 0)
adding edge.... 56->16 (30, 0)
adding edge.... 16->56 (30, 0)
adding edge.... 36->26 (34, 0)
adding edge.... 26->36 (34, 0)
adding edge.... 24->13 (35, 30)
adding edge.... 13->24 (35, 30)
adding edge.... 36->34 (25, 0)
adding edge.... 34->36 (25, 0)
adding edge.... 32->16 (100, 50)
adding edge.... 16->32 (100, 50)
adding edge.... 32->12 

.

Loading map from file...
adding edge.... 32->36 (70, 0)
adding edge.... 36->32 (70, 0)
adding edge.... 32->57 (30, 0)
adding edge.... 57->32 (30, 0)
adding edge.... 32->76 (80, 50)
adding edge.... 76->32 (80, 50)
adding edge.... 32->68 (110, 80)
adding edge.... 68->32 (110, 80)
adding edge.... 76->68 (72, 30)
adding edge.... 68->76 (72, 30)
adding edge.... 68->66 (51, 0)
adding edge.... 66->68 (51, 0)
adding edge.... 68->56 (80, 70)
adding edge.... 56->68 (80, 70)
adding edge.... 66->56 (40, 0)
adding edge.... 56->66 (40, 0)
adding edge.... 76->66 (130, 100)
adding edge.... 66->76 (130, 100)
adding edge.... 56->18 (35, 0)
adding edge.... 18->56 (35, 0)
adding edge.... 56->16 (30, 0)
adding edge.... 16->56 (30, 0)
adding edge.... 36->26 (34, 0)
adding edge.... 26->36 (34, 0)
adding edge.... 24->13 (35, 30)
adding edge.... 13->24 (35, 30)
adding edge.... 36->34 (25, 0)
adding edge.... 34->36 (25, 0)
adding edge.... 32->16 (100, 50)
adding edge.... 16->32 (100, 50)
adding edge.... 32->12 

.

...

Loading map from file...
adding edge.... 32->36 (70, 0)
adding edge.... 36->32 (70, 0)
adding edge.... 32->57 (30, 0)
adding edge.... 57->32 (30, 0)
adding edge.... 32->76 (80, 50)
adding edge.... 76->32 (80, 50)
adding edge.... 32->68 (110, 80)
adding edge.... 68->32 (110, 80)
adding edge.... 76->68 (72, 30)
adding edge.... 68->76 (72, 30)
adding edge.... 68->66 (51, 0)
adding edge.... 66->68 (51, 0)
adding edge.... 68->56 (80, 70)
adding edge.... 56->68 (80, 70)
adding edge.... 66->56 (40, 0)
adding edge.... 56->66 (40, 0)
adding edge.... 76->66 (130, 100)
adding edge.... 66->76 (130, 100)
adding edge.... 56->18 (35, 0)
adding edge.... 18->56 (35, 0)
adding edge.... 56->16 (30, 0)
adding edge.... 16->56 (30, 0)
adding edge.... 36->26 (34, 0)
adding edge.... 26->36 (34, 0)
adding edge.... 24->13 (35, 30)
adding edge.... 13->24 (35, 30)
adding edge.... 36->34 (25, 0)
adding edge.... 34->36 (25, 0)
adding edge.... 32->16 (100, 50)
adding edge.... 16->32 (100, 50)
adding edge.... 32->12 

.

Loading map from file...
adding edge.... 32->36 (70, 0)
adding edge.... 36->32 (70, 0)
adding edge.... 32->57 (30, 0)
adding edge.... 57->32 (30, 0)
adding edge.... 32->76 (80, 50)
adding edge.... 76->32 (80, 50)
adding edge.... 32->68 (110, 80)
adding edge.... 68->32 (110, 80)
adding edge.... 76->68 (72, 30)
adding edge.... 68->76 (72, 30)
adding edge.... 68->66 (51, 0)
adding edge.... 66->68 (51, 0)
adding edge.... 68->56 (80, 70)
adding edge.... 56->68 (80, 70)
adding edge.... 66->56 (40, 0)
adding edge.... 56->66 (40, 0)
adding edge.... 76->66 (130, 100)
adding edge.... 66->76 (130, 100)
adding edge.... 56->18 (35, 0)
adding edge.... 18->56 (35, 0)
adding edge.... 56->16 (30, 0)
adding edge.... 16->56 (30, 0)
adding edge.... 36->26 (34, 0)
adding edge.... 26->36 (34, 0)
adding edge.... 24->13 (35, 30)
adding edge.... 13->24 (35, 30)
adding edge.... 36->34 (25, 0)
adding edge.... 34->36 (25, 0)
adding edge.... 32->16 (100, 50)
adding edge.... 16->32 (100, 50)
adding edge.... 32->12 

.
----------------------------------------------------------------------
Ran 9 tests in 4.463s

OK


Loading map from file...
adding edge.... 32->36 (70, 0)
adding edge.... 36->32 (70, 0)
adding edge.... 32->57 (30, 0)
adding edge.... 57->32 (30, 0)
adding edge.... 32->76 (80, 50)
adding edge.... 76->32 (80, 50)
adding edge.... 32->68 (110, 80)
adding edge.... 68->32 (110, 80)
adding edge.... 76->68 (72, 30)
adding edge.... 68->76 (72, 30)
adding edge.... 68->66 (51, 0)
adding edge.... 66->68 (51, 0)
adding edge.... 68->56 (80, 70)
adding edge.... 56->68 (80, 70)
adding edge.... 66->56 (40, 0)
adding edge.... 56->66 (40, 0)
adding edge.... 76->66 (130, 100)
adding edge.... 66->76 (130, 100)
adding edge.... 56->18 (35, 0)
adding edge.... 18->56 (35, 0)
adding edge.... 56->16 (30, 0)
adding edge.... 16->56 (30, 0)
adding edge.... 36->26 (34, 0)
adding edge.... 26->36 (34, 0)
adding edge.... 24->13 (35, 30)
adding edge.... 13->24 (35, 30)
adding edge.... 36->34 (25, 0)
adding edge.... 34->36 (25, 0)
adding edge.... 32->16 (100, 50)
adding edge.... 16->32 (100, 50)
adding edge.... 32->12 